In [ ]:
# to cite the article : RBF "have low confidence when they are fooled", 
# It has low precision but "it does correctly respond by reducing its confidence considerably on points it does not 'understand'."
# Result to reproduce : 
# Shallow RBF network with no hidden layers : 
#            - confidence on mistaken examples = 1.2%
#            - confidence on clean test examples = 60.6%
# We will use the MNSIT dataset

In [6]:
# Importations
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_openml

import torch
import torch.nn as nn
import torch.nn.functional as F

# Import the functions from fgsm.py
from fgsm_copie import adv_attack, test



In [21]:

# Load the MNIST dataset
mnist = fetch_openml('mnist_784')

# Print the dataset information
#print(mnist.DESCR)

# Access the features and target variables
X = mnist.data
y = mnist.target

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [51]:
# We convert the data into numpy array (cause for now they are in pandas)
X1 = np.array(X)
y1 = np.array(y)

# the labels y are string, we convert them into float32
y1 = y1.astype(np.float32)


print("X1 shape : ", X1.shape)
print("y1 shape : ", y1.shape)
print("X1 type : ", type(X1[0][0]))
print("y1 type : ", type(y1[0]))
print(y1)

X1 shape :  (70000, 784)
y1 shape :  (70000,)
X1 type :  <class 'numpy.float32'>
y1 type :  <class 'numpy.float32'>
[5. 0. 4. ... 4. 5. 6.]


In [49]:

# There we create the class for our Shallow RBF network

class ShallowRBF(nn.Module):
    def __init__(self, input_dim, num_classes, num_centers):
        super(ShallowRBF, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_centers, input_dim))
        self.beta = nn.Parameter(torch.ones(num_centers))
        self.fc = nn.Linear(num_centers, num_classes)

    def forward(self, x):
        # Calculate the RBF activations
        rbf_activations = torch.exp(-self.beta * torch.norm(x.unsqueeze(1) - self.centers, dim=2))

        # Normalize the RBF activations
        rbf_activations = rbf_activations / torch.sum(rbf_activations, dim=1, keepdim=True)

        # Pass the normalized RBF activations through the linear layer
        output = self.fc(rbf_activations)

        return output


In [54]:
# Set the hyperparameters
input_dim = 784
num_classes = 10
num_centers = 100
num_epochs = 10
batch_size = 100
learning_rate = 0.1

epsilon = 0.3
device = torch.device('cpu')

In [53]:

# We will now train our Shallow RBF network

# Initialize the model
model = ShallowRBF(input_dim, num_classes, num_centers)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Convert the data into PyTorch tensors
X = torch.from_numpy(X1).float()
y = torch.squeeze(torch.from_numpy(y1).long())

# Create the dataloader
dataset = torch.utils.data.TensorDataset(X, y)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Train the model
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(dataloader):
        # Forward pass
        output = model(data)

        # Calculate the loss
        loss = criterion(output, target)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the progress
        if (batch_idx + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, batch_idx + 1, len(dataloader), loss.item()))
            
# We will now test our Shallow RBF network

# Set the model in evaluation mode
model.eval()

#test( model, device, normal_loader, epsilon )

# Calculate the accuracy on the clean test set
clean_acc = test(model, device, dataloader, epsilon)
print('Accuracy on clean examples: {:.2f}%'.format(clean_acc))

# Calculate the accuracy on the adversarial test set
adversarial_acc = test(model, device, dataloader, epsilon)
print('Accuracy on adversarial examples: {:.2f}%'.format(adversarial_acc))

# Calculate the confidence of the model on the clean test set
clean_confidence = test(model, dataloader, return_confidence=True)
print('Average confidence on clean examples: {:.2f}%'.format(clean_confidence))

# Calculate the confidence of the model on the adversarial test set
adversarial_confidence = test(model, dataloader, adv_attack, return_confidence=True)
print('Average confidence on adversarial examples: {:.2f}%'.format(adversarial_confidence))


Epoch [1/10], Step [100/700], Loss: nan
Epoch [1/10], Step [200/700], Loss: nan
Epoch [1/10], Step [300/700], Loss: nan
Epoch [1/10], Step [400/700], Loss: nan
Epoch [1/10], Step [500/700], Loss: nan
Epoch [1/10], Step [600/700], Loss: nan
Epoch [1/10], Step [700/700], Loss: nan
Epoch [2/10], Step [100/700], Loss: nan
Epoch [2/10], Step [200/700], Loss: nan
Epoch [2/10], Step [300/700], Loss: nan
Epoch [2/10], Step [400/700], Loss: nan
Epoch [2/10], Step [500/700], Loss: nan
Epoch [2/10], Step [600/700], Loss: nan
Epoch [2/10], Step [700/700], Loss: nan
Epoch [3/10], Step [100/700], Loss: nan
Epoch [3/10], Step [200/700], Loss: nan
Epoch [3/10], Step [300/700], Loss: nan
Epoch [3/10], Step [400/700], Loss: nan
Epoch [3/10], Step [500/700], Loss: nan
Epoch [3/10], Step [600/700], Loss: nan
Epoch [3/10], Step [700/700], Loss: nan
Epoch [4/10], Step [100/700], Loss: nan
Epoch [4/10], Step [200/700], Loss: nan
Epoch [4/10], Step [300/700], Loss: nan
Epoch [4/10], Step [400/700], Loss: nan


TypeError: test() missing 2 required positional arguments: 'normal_loader' and 'epsilon'